In [66]:
import torch
from torch import tensor, matmul, FloatTensor

def layer(x):
    w = tensor([[1.,2.],[3.,-4.]])
    b = tensor([[1.],[2.]])
    return torch.relu(matmul(w, x) + b)

In [69]:
x = tensor([[-1],[1]], dtype=torch.float)
x.requires_grad_()
output = torch.sum(matmul(tensor([1.,-1.]), layer(x)))
output.backward()
print(output)
x.grad

tensor(2., grad_fn=<SumBackward0>)


tensor([[1.],
        [2.]])

In [ ]:
# Input 
# x = .25 + .01eps1
# y = .1 + .01eps2


$\text{Input Domain}: L_\infty$

\begin{equation}
\psi : \left( \begin{array}{c}x \\ y \\ \end{array} \right)=
\left( \begin{array}{c}x_0 \\ y_0 \\ \end{array} \right)
+\epsilon_1\left( \begin{array}{c} \eta \\ 0 \\ \end{array} \right)
+\epsilon_2 \left( \begin{array}{c}0 \\ \eta \\ \end{array} \right)
\end{equation}
$,\,\forall\,i,\,-1\leq\epsilon_i\leq1$

In [94]:
x0, y0, eta = 0., 0., 1.
xy1 = tensor([[x0, eta, 0], [y0, 0, eta]])

In [102]:
# Compute affine transform 
# x3, x4 = -x1 + x2 + 1, 2x1 - x2 + 2
w = torch.tensor([[-1, 1],[2, -1]], dtype=torch.float)
b = torch.tensor([[1,0,0],[2,0,0]], dtype=torch.float)
x3, x4 = matmul(w ,xy1) + b

Affine transform

\begin{equation}
\left(
\begin{array}{cc}
 w_{1,1} & w_{1,2} \\
 w_{2,1} & w_{2,2} \\
\end{array}
\right).\left(
\begin{array}{ccc}
 x_0 & \eta  & 0 \\
 y_0 & 0 & \eta  \\
\end{array}
\right)+\left(
\begin{array}{c}
 b_1 & 0 & 0 \\
 b_2 & 0 & 0\\
\end{array}
\right)
\end{equation}

In [100]:
def box(x):
    radius = torch.sum(torch.abs(x[1:]))
    return x[0] - radius, x[0] + radius

\begin{equation}ReLU^{\#}(x)=
\lambda  x+ \frac{1}{2} \left(\epsilon _{\text{new}}+1\right)\begin{cases}
 -l \lambda  & \lambda >\frac{u}{u-l} \\
 u (1-\lambda ) & \text{Otherwise} \\
\end{cases}
\end{equation}

In [114]:
def relu_transformer(x, lmb, epsilon_id = None):
    l, u = box(x)
    if u <= 0:
        return torch.zeros(len(x))
    elif l >= 0:
        return x
    else:
        if epsilon_id is None:
            epsilon_id = len(x)
        x = torch.nn.ConstantPad1d((0, epsilon_id - len(x) + 1),0)(x)
        x *= lmb
        if lmb >= u/(u-1):
            x[epsilon_id] = -l * lmb / 2
        else:
            x[epsilon_id] = u * (1 - lmb)
        
        x[0] += x[epsilon_id]
        return x

In [128]:
x = tensor([1,1, -1])
print(box(x))
lmb = torch.tensor(1/3).float()
lmb.requires_grad_()
y = torch.sum(relu_transformer(x, lmb))
y.backward()
print(lmb.grad)

(tensor(1), tensor(1))


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn